<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/tutorials/quickstart/hybrid_search_with_milvus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/tutorials/quickstart/hybrid_search_with_milvus.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>

# Hybrid Search với Vector Dense và Sparse trong Milvus
Để trải nghiệm kết quả cuối cùng của hướng dẫn này, có thể truy cập trực tiếp tại: https://demos.milvus.io/hybrid-search/

<img src="https://raw.githubusercontent.com/milvus-io/bootcamp/master/tutorials/quickstart/apps/hybrid_demo_with_milvus/pics/demo.png"/>

Hướng dẫn này trình bày cách thực hiện **hybrid search** với [Milvus](https://milvus.io/docs/multi-vector-search.md) và mô hình **BGE-M3**. Mô hình BGE-M3 có thể chuyển văn bản thành **dense vector** và **sparse vector**. Milvus hỗ trợ lưu cả hai loại vector trong một collection, cho phép hybrid search nhằm tăng độ liên quan của kết quả.

Milvus hỗ trợ các phương pháp truy hồi: **Dense**, **Sparse**, và **Hybrid**:

- **Dense Retrieval**: Tận dụng ngữ cảnh ngữ nghĩa để hiểu ý nghĩa phía sau truy vấn.
- **Sparse Retrieval**: Nhấn mạnh khớp từ khóa để tìm kết quả dựa trên các thuật ngữ cụ thể, tương đương full-text search.
- **Hybrid Retrieval**: Kết hợp cả Dense và Sparse, vừa nắm bắt ngữ cảnh tổng thể vừa giữ được các từ khóa cụ thể để có kết quả toàn diện.

Bằng cách tích hợp các phương pháp này, **Milvus Hybrid Search** cân bằng độ tương đồng ngữ nghĩa và từ vựng, cải thiện độ liên quan tổng thể của kết quả tìm kiếm. Notebook này sẽ hướng dẫn quy trình thiết lập và sử dụng các chiến lược truy hồi, làm nổi bật hiệu quả của chúng trong nhiều kịch bản tìm kiếm.

### Dependencies and Environment

In [1]:
!pip install --upgrade "pymilvus[model,milvus_lite]"

You should consider upgrading via the 'C:\Users\vietcq\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


### Download Dataset

Để minh họa tìm kiếm, cần một tập tài liệu (corpus). Tập dữ liệu **Quora Duplicate Questions** sẽ được sử dụng và đặt vào thư mục cục bộ.

Nguồn dữ liệu: **First Quora Dataset Release: Question Pairs**

In [2]:
# Run this cell to download the dataset
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

'wget' is not recognized as an internal or external command,
operable program or batch file.


### Load and Prepare Data

Tải dữ liệu và chuẩn bị một corpus nhỏ để tìm kiếm.

In [3]:
import pandas as pd

file_path = "quora_duplicate_questions.tsv"
df = pd.read_csv(file_path, sep="\t")
questions = set()
for _, row in df.iterrows():
    obj = row.to_dict()
    questions.add(obj["question1"][:512])
    questions.add(obj["question2"][:512])
    if len(questions) > 500:  # Skip this if you want to use the full dataset
        break

docs = list(questions)

# example question
print(docs[0])

What would a Trump presidency mean for current international master’s students on an F1 visa?


### Use BGE-M3 Model for Embeddings

Mô hình **BGE-M3** có thể tạo embedding văn bản dưới dạng **dense vector** và **sparse vector**.

In [4]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(docs)

c:\Users\vietcq\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 32/32 [00:00<00:00, 1364.11it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
pre tokenize: 100%|██████████| 32/32 [00:00<00:00, 1364.11it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 32/32 [00:22<00:00,  1.42it/s]

### Setup Milvus Collection and Index

Thiết lập collection trong Milvus và tạo index cho các trường vector.

- Việc đặt **uri** là file cục bộ, ví dụ `./milvus.db`, là cách tiện lợi nhất, vì nó tự động dùng **Milvus Lite** để lưu toàn bộ dữ liệu vào một file này.
- Với dữ liệu quy mô lớn, ví dụ hơn một triệu vector, có thể thiết lập Milvus server có hiệu năng tốt hơn trên **Docker** hoặc **Kubernetes**. Trong thiết lập này, hãy dùng **server uri**, ví dụ `http://localhost:19530`, làm `uri`.
- Để dùng **Zilliz Cloud** (dịch vụ đám mây được quản lý toàn phần cho Milvus), cần điều chỉnh `uri` và `token`, tương ứng với **Public Endpoint** và **API key** trong Zilliz Cloud.

In [5]:
# Cài đặt thư viện fallback nếu cần
try:
    from sklearn.metrics.pairwise import cosine_similarity
    from scipy.sparse import csr_matrix
    import numpy as np
except ImportError:
    %pip install scikit-learn scipy numpy

from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

# Tạo SimpleVectorStore cho Windows (fallback solution)
class SimpleVectorStore:
    def __init__(self):
        self.texts = []
        self.dense_vectors = []
        self.sparse_vectors = []
        self.num_entities = 0
    
    def insert(self, batched_entities):
        texts, sparse_vecs, dense_vecs = batched_entities
        self.texts.extend(texts)
        self.dense_vectors.extend(dense_vecs)
        self.sparse_vectors.extend(sparse_vecs)
        self.num_entities = len(self.texts)

USE_MILVUS = False
col = None

# Thử kết nối Milvus Lite
try:
    connections.connect(uri="./milvus.db")
    print("✓ Kết nối thành công với Milvus Lite!")
    USE_MILVUS = True
    
    # Specify the data schema for the new Collection
    fields = [
        FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
        FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
        FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
    ]
    schema = CollectionSchema(fields)

    col_name = "hybrid_demo"
    if utility.has_collection(col_name):
        Collection(col_name).drop()
    
    col = Collection(col_name, schema)
    sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
    col.create_index("sparse_vector", sparse_index)
    dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
    col.create_index("dense_vector", dense_index)
    col.load()
    
except Exception as e:
    print(f"Milvus Lite không hoạt động: {str(e)[:100]}...")
    print("Tự động chuyển sang SimpleVectorStore (hoạt động trên Windows)")
    USE_MILVUS = False
    col = SimpleVectorStore()

print(f"Backend được sử dụng: {'Milvus' if USE_MILVUS else 'SimpleVectorStore'}")

Milvus Lite không hoạt động: <ConnectionConfigException: (code=1, message=milvus-lite is required for local database connections....
Tự động chuyển sang SimpleVectorStore (hoạt động trên Windows)
Backend được sử dụng: SimpleVectorStore


### Insert Data into Milvus Collection

Chèn các document và embedding của chúng vào collection.

In [6]:
# Chèn dữ liệu vào vector store
if USE_MILVUS:
    # For efficiency, we insert 50 records in each small batch for Milvus
    for i in range(0, len(docs), 50):
        batched_entities = [
            docs[i : i + 50],
            docs_embeddings["sparse"][i : i + 50],
            docs_embeddings["dense"][i : i + 50],
        ]
        col.insert(batched_entities)
    print("Number of entities inserted:", col.num_entities)
else:
    # Chèn toàn bộ dữ liệu cho SimpleVectorStore
    batched_entities = [docs, docs_embeddings["sparse"], docs_embeddings["dense"]]
    col.insert(batched_entities)
    print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 502


### Enter Your Search Query

In [7]:
# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
# print(query_embeddings)

How to start learning programming?


### Run the Search

Đầu tiên, cần chuẩn bị vài hàm tiện ích để chạy tìm kiếm:

- `dense_search`: chỉ tìm trên trường **dense vector**
- `sparse_search`: chỉ tìm trên trường **sparse vector**
- `hybrid_search`: tìm trên cả hai trường vector với **weighted reranker**

In [8]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

def dense_search(col, query_dense_embedding, limit=10):
    if USE_MILVUS:
        search_params = {"metric_type": "IP", "params": {}}
        res = col.search(
            [query_dense_embedding],
            anns_field="dense_vector",
            limit=limit,
            output_fields=["text"],
            param=search_params,
        )[0]
        return [hit.get("text") for hit in res]
    else:
        # SimpleVectorStore implementation
        dense_matrix = np.array(col.dense_vectors)
        query_vec = np.array(query_dense_embedding).reshape(1, -1)
        similarities = cosine_similarity(query_vec, dense_matrix)[0]
        top_indices = np.argsort(similarities)[::-1][:limit]
        return [col.texts[idx] for idx in top_indices]

def sparse_search(col, query_sparse_embedding, limit=10):
    if USE_MILVUS:
        search_params = {"metric_type": "IP", "params": {}}
        res = col.search(
            [query_sparse_embedding],
            anns_field="sparse_vector",
            limit=limit,
            output_fields=["text"],
            param=search_params,
        )[0]
        return [hit.get("text") for hit in res]
    else:
        # SimpleVectorStore implementation - sửa xử lý sparse vector
        similarities = []
        
        # Xử lý query sparse vector
        if hasattr(query_sparse_embedding, 'indices') and hasattr(query_sparse_embedding, 'values'):
            # Đây là sparse vector từ BGE-M3
            query_dict = dict(zip(query_sparse_embedding.indices, query_sparse_embedding.values))
        else:
            # Fallback: chuyển đổi thành dict
            query_dict = {}
            if hasattr(query_sparse_embedding, 'toarray'):
                dense_query = query_sparse_embedding.toarray().flatten()
                query_dict = {i: val for i, val in enumerate(dense_query) if val != 0}
            
        for sparse_vec in col.sparse_vectors:
            # Xử lý document sparse vector tương tự
            if hasattr(sparse_vec, 'indices') and hasattr(sparse_vec, 'values'):
                doc_dict = dict(zip(sparse_vec.indices, sparse_vec.values))
            else:
                doc_dict = {}
                if hasattr(sparse_vec, 'toarray'):
                    dense_doc = sparse_vec.toarray().flatten()
                    doc_dict = {i: val for i, val in enumerate(dense_doc) if val != 0}
            
            # Tính inner product (IP) giữa hai sparse vector
            sim = sum(query_dict.get(idx, 0) * doc_dict.get(idx, 0) 
                     for idx in set(query_dict.keys()) | set(doc_dict.keys()))
            similarities.append(sim)
        
        similarities = np.array(similarities)
        top_indices = np.argsort(similarities)[::-1][:limit]
        return [col.texts[idx] for idx in top_indices]

def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    if USE_MILVUS:
        dense_search_params = {"metric_type": "IP", "params": {}}
        dense_req = AnnSearchRequest(
            [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
        )
        sparse_search_params = {"metric_type": "IP", "params": {}}
        sparse_req = AnnSearchRequest(
            [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
        )
        rerank = WeightedRanker(sparse_weight, dense_weight)
        res = col.hybrid_search(
            [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
        )[0]
        return [hit.get("text") for hit in res]
    else:
        # SimpleVectorStore implementation - combine dense and sparse scores
        dense_results = dense_search(col, query_dense_embedding, limit*2)
        sparse_results = sparse_search(col, query_sparse_embedding, limit*2)
        
        # Combine scores với weighted ranking
        text_scores = {}
        
        # Thêm dense scores với weight
        for i, text in enumerate(dense_results):
            score = dense_weight * (1.0 - i / len(dense_results))  # Giảm dần theo thứ hạng
            text_scores[text] = text_scores.get(text, 0) + score
        
        # Thêm sparse scores với weight
        for i, text in enumerate(sparse_results):
            score = sparse_weight * (1.0 - i / len(sparse_results))  # Giảm dần theo thứ hạng
            text_scores[text] = text_scores.get(text, 0) + score
            
        # Sort by combined scores and return top results
        sorted_texts = sorted(text_scores.items(), key=lambda x: x[1], reverse=True)[:limit]
        return [text for text, score in sorted_texts]

Chạy ba loại tìm kiếm với các hàm đã định nghĩa:

In [9]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][0]) 
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][0],
    sparse_weight=0.7,
    dense_weight=1.0,
)

### Display Search Results

Để hiển thị kết quả của Dense, Sparse và Hybrid, cần có vài tiện ích để định dạng kết quả.

In [10]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

Sau đó, có thể hiển thị kết quả ở dạng văn bản kèm highlight:

In [11]:
from IPython.display import Markdown, display

# Dense search results
display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

# Sparse search results
display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))

# Hybrid search results
display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))

**Dense Search Results:**

What's the best way to start learning robotics?

How do I learn a computer language like java?

How can I get started to learn information security?

What is Java programming? How To Learn Java Programming Language ?

...

What math does a complete newbie need<span style='color:red'> to</span> understand algorithms for computer<span style='color:red'> programming?</span> What books on algorithms are suitable for a complete beginner<span style='color:red'>?</span>

<span style='color:red'>How</span> can I learn computer security<span style='color:red'>?</span>

<span style='color:red'>How</span> can you make physics easy<span style='color:red'> to</span> learn<span style='color:red'>?</span>

<span style='color:red'>How</span> can I learn<span style='color:red'> to</span> speak English fluently<span style='color:red'>?</span>

What are the best ways<span style='color:red'> to</span> learn French<span style='color:red'>?</span>

### Quick Deploy

Để tìm hiểu về cách bắt đầu một bản demo trực tuyến với hướng dẫn này, vui lòng tham khảo [ứng dụng ví dụ](https://github.com/milvus-io/bootcamp/tree/master/tutorials/quickstart/apps/hybrid_demo_with_milvus).